In [1]:
import logging
import json
from typing import List
import re

In [14]:
def check_response_crossover(response: str, parents: List[str]) -> List[str]:
    """
    解析 LLM 生成的 `response`，提取 `expressions`，如果解析失败，则返回 `parents` 作为备选值。
    增加调试信息输出，帮助追踪中间变量。
    """
    try:
        print("\n📥 原始 response：")
        print(response)

        response_cleaned = re.sub(r'```json\n|```', '', response).replace('\n', '')
        print("\n📃 清洗后的 response_cleaned：")
        print(response_cleaned)

        match = re.search(r'\"expressions\"\s*:\s*\[(.*?)\]', response_cleaned, re.DOTALL)
        if not match:
            print("❌ 未找到 expressions 字段，返回 parents")
            return parents

        expressions_string = match.group(1)
        print("\n🧩 提取出的 expressions_string：")
        print(expressions_string)

        # 尝试提取 {} 包裹的结构（可能是 {"expr": "..."} 的格式）
        expressions = re.findall(r'\{([^\}]+)\}', expressions_string)
        if expressions:
            cleaned_exprs = [expr.strip().replace('"', '') for expr in expressions]
            print("\n✅ 从大括号中提取的表达式 cleaned_exprs：")
            print(cleaned_exprs)
            if len(cleaned_exprs) == 2:
                return cleaned_exprs
            else:
                print("⚠️ 提取数量不足 2 个，返回 parents")
                return parents

        # 尝试用逗号分割
        final_exprs = [expr.strip().replace('"', '').replace("{", "").replace("}", "")
                       for expr in re.split(r'\s*,\s*', expressions_string)]

        print("\n🔍 逗号分割方式得到的 final_exprs：")
        print(final_exprs)

        if len(final_exprs) == 2:
            print("✅ 成功解析出两个表达式")
            return final_exprs
        else:
            print("⚠️ 表达式数量不足 2 个，返回 parents")
            return parents

    except (ValueError, json.JSONDecodeError) as e:
        logging.error(f"解析 LLM 交叉变异响应失败: {e}")
        print("🚨 解析异常，返回 parents")
        return parents


In [18]:
import re
import logging
from typing import List

def check_response_crossover(response: str, parents: List[str]) -> List[str]:
    """
    健壮地从 LLM 的非标准 JSON 响应中提取 expressions（字符串列表），
    避免使用 split(',') 导致的表达式拆分错误。
    """
    try:
        print("\n📥 原始 response：")
        print(response)

        # Step 1: 清洗 markdown 包装
        response_cleaned = re.sub(r'```json\n|```', '', response).strip()
        print("\n📃 清洗后的 response_cleaned：")
        print(response_cleaned)

        # Step 2: 提取 expressions 字符串部分
        match = re.search(r'"expressions"\s*:\s*\[(.*?)\]', response_cleaned, re.DOTALL)
        if not match:
            print("❌ 未能匹配 expressions 字段")
            return parents

        expressions_block = match.group(1)
        print("\n🧩 提取出的 expressions_block：")
        print(expressions_block)

        # Step 3: 用正则提取每个被引号包裹的完整表达式
        expr_matches = re.findall(r'"(.*?)"', expressions_block)
        print("\n🧪 提取出的 expression 列表：")
        for idx, expr in enumerate(expr_matches):
            print(f"  Expression {idx+1}: {expr}")

        return expr_matches if len(expr_matches) == 2 else parents

    except Exception as e:
        logging.error(f"解析 LLM 交叉表达式失败: {e}")
        print(f"🚨 解析出错: {e}")
        return parents


In [19]:
def test_check_response_crossover(response: str, parents: List[str]):
    """
    用于测试 check_response_crossover() 函数的实际表现。
    打印输入、输出、并检查是否合理。
    """
    print("🧪 ====== 测试 LLM 交叉响应解析 ======")
    print("📥 原始 response:")
    print(response)
    print("👨‍👩‍👧‍👦 parents:")
    print(parents)

    result = check_response_crossover(response, parents)

    print("✅ 最终解析结果:")
    print(result)

    # if result == parents:
    #     print("⚠️ 使用了默认 parents（说明 response 结构可能不匹配或解析失败）")
    # elif len(result) != 2:
    #     print("❌ 错误：返回表达式数量不为 2")
    # else:
    #     print("🎉 成功提取 2 个表达式！")


In [20]:
test_response = '{\n  "expressions": [\n    "min(ACT + ACT - TCT / TCT * max(TPS, TNC) / TWT / TWT, TWT - TPS - max(TPS, TNC) - TCT + ACT / max(TWT, ACT))",\n    "min(max(min(TPS / TNC, TCT + TCT), TCT + TNC * max(TNC, TCT)), -(-(TWT)) - min(TCT, TPS) - TWT / ACT)"\n  ]\n}'
parents = [
    "add(TCT, mul(TWT, TNC))",
    "min(TNC, protect_div(TUT, TCT))"
]

In [21]:

test_check_response_crossover(test_response, parents)

🧪 ====== 测试 LLM 交叉响应解析 ======
📥 原始 response:
{
  "expressions": [
    "min(ACT + ACT - TCT / TCT * max(TPS, TNC) / TWT / TWT, TWT - TPS - max(TPS, TNC) - TCT + ACT / max(TWT, ACT))",
    "min(max(min(TPS / TNC, TCT + TCT), TCT + TNC * max(TNC, TCT)), -(-(TWT)) - min(TCT, TPS) - TWT / ACT)"
  ]
}
👨‍👩‍👧‍👦 parents:
['add(TCT, mul(TWT, TNC))', 'min(TNC, protect_div(TUT, TCT))']

📥 原始 response：
{
  "expressions": [
    "min(ACT + ACT - TCT / TCT * max(TPS, TNC) / TWT / TWT, TWT - TPS - max(TPS, TNC) - TCT + ACT / max(TWT, ACT))",
    "min(max(min(TPS / TNC, TCT + TCT), TCT + TNC * max(TNC, TCT)), -(-(TWT)) - min(TCT, TPS) - TWT / ACT)"
  ]
}

📃 清洗后的 response_cleaned：
{
  "expressions": [
    "min(ACT + ACT - TCT / TCT * max(TPS, TNC) / TWT / TWT, TWT - TPS - max(TPS, TNC) - TCT + ACT / max(TWT, ACT))",
    "min(max(min(TPS / TNC, TCT + TCT), TCT + TNC * max(TNC, TCT)), -(-(TWT)) - min(TCT, TPS) - TWT / ACT)"
  ]
}

🧩 提取出的 expressions_block：

    "min(ACT + ACT - TCT / TCT * max(TPS, TNC) / 